In [1]:
import math
import torch.nn as nn
import torch.nn.functional as F
import pdb
import matplotlib.pyplot as plt
 
import random
import torch

# 加载数据

## 初始化参数设置

In [2]:
layers = [1, 2, 4, 1]
num_classes = 2
zero_init_residual=True
groups=8
width_per_group=64
norm_layer=nn.BatchNorm2d
s=0.125
img_size=128
imgchan=3
batchsize=2
replace_stride_with_dilation=None

In [3]:
inplanes = int(64 * s)
dilation = 1
if replace_stride_with_dilation is None:
    replace_stride_with_dilation = [False, False, False]
base_width = width_per_group
conv1 = nn.Conv2d(imgchan, inplanes, kernel_size=7, stride=2, padding=3, bias=False)
conv2 = nn.Conv2d(inplanes, 128, kernel_size=3, stride=1, padding=1,bias=False)
conv3 = nn.Conv2d(128, inplanes, kernel_size=3, stride=1, padding=1, bias=False)
bn1 = norm_layer(inplanes)
bn2 = norm_layer(128)
bn3 = norm_layer(inplanes)
relu = nn.ReLU(inplace=True)

In [5]:
x = torch.randn(batchsize, imgchan, img_size, img_size)
xin = x.clone()
print(x.shape)

torch.Size([2, 3, 128, 128])


## 初始化卷积

In [7]:
x1 = relu(bn1(conv1(x)))
x2 = relu(bn2(conv2(x1)))
x3 = relu(bn3(conv3(x2)))
print(x.shape)

torch.Size([2, 3, 128, 128])


In [9]:
# self.layer1 = self._make_layer(block, int(128 * s), layers[0], kernel_size= (img_size//2))
planes = int(128 * s)
dilate = False
stride = 1
kernel_size = (img_size//2)
downsample = None
previous_dilation = dilation
if dilate:
    dilation *= stride
    stride = 1


# block(self.inplanes, planes, stride, downsample, groups=self.groups,
#                             base_width=self.base_width, dilation=previous_dilation, 
#                             norm_layer=norm_layer, kernel_size=kernel_size)

# AxialAttention

## 初始化参数设置

In [10]:
width=False
in_planes = inplanes
out_planes = planes
group_planes = out_planes // groups
qkv_transform = nn.Conv1d(in_planes, out_planes * 2, kernel_size=1, stride=1, padding=0, bias=False)
bn_qkv = nn.BatchNorm1d(out_planes * 2)
bn_similarity = nn.BatchNorm2d(groups * 3)
bn_output = nn.BatchNorm1d(out_planes * 2)

## 位置编码

In [11]:
relative = nn.Parameter(torch.randn(group_planes * 2, kernel_size *2 -1), requires_grad=True)
query_index = torch.arange(kernel_size).unsqueeze(0)
key_index = torch.arange(kernel_size).unsqueeze(1)
relative_index = key_index - query_index + kernel_size - 1

flatten_index = nn.Parameter(relative_index.view(-1), requires_grad=False)

In [13]:
xia = x3
print("xia befroe permute",xia.shape)
xia = xia.permute(0, 3, 1, 2)
print("xia after permute",xia.shape)

N, W, C, H = xia.shape
xia = xia.contiguous().view(N * W, C, H)
print("xia after view(N*W)",xia.shape)

xia befroe permute torch.Size([2, 8, 64, 64])
xia after permute torch.Size([2, 64, 8, 64])
xia after view(N*W) torch.Size([128, 8, 64])


In [14]:
qkv = bn_qkv(qkv_transform(xia))
print(qkv.shape)

torch.Size([128, 32, 64])
